## A bit of prep..
To make sure you have all the libraries, run the following command in the `Anaconda Prompt`
```bash
conda install -c conda-forge geopandas decartes
```

to download the boundary layer needed for the map you can run the following code:
```python
import urllib.request

url = 'https://opendata.arcgis.com/datasets/2c5b8eb836c7475ba3b305106ac9dfc3_0.geojson'
urllib.request.urlretrieve(url, './data/bounderies.geojson')
```

# Let's Map!
We are going to need 2 data sources:
* **Implementation date data**: in the *UCimplem.csv* (souce: Department for Work and Pensions)
* **Geolocated boundery file for all local authorities**: Can be downloaded from the Office for National Statistics: https://geoportal.statistics.gov.uk/datasets/local-authority-districts-december-2016-boundaries (we have downloaded the ultra generalised layer)

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from filled_out.compatibility import ensure_mutipoly
%matplotlib inline

#### Implementation Data Loading

In [ ]:
# load 'data/UCimplem.csv'

In [ ]:
# describe the dataset

### Column renaming
The column name is misleading, the `County` are actually local authorities. Let's rename the colum!

In [ ]:
# rename 'County' to 'LocalAuth'

### Date parsing

In [ ]:
def date_parser(x):
    return pd.to_datetime(x, format='%b-%y').to_datetime64().astype(float)

# apply it to 'UC implem' and call 'UC_date_number'

In [ ]:
# check head, 2 rows

### Boundary Layers Loading

In [ ]:
# load 'data/bounderies.geojson' with geopandas's "read_file" method onto a variable called "gdf"
# check head, 2 rows

### Merging the data

In [ ]:
merged_gdf = gdf.merge(
    right=df, 
    left_on='lad18cd', 
    right_on='Code',
    how='outer'
)

In [ ]:
cmap = 'Blues'
ax = merged_gdf.plot(
    column='UC_date_number', 
    linewidth=0.3, edgecolor='black', 
    figsize=(18, 12), 
    cmap=cmap,
)

# to change color select a different cmap from here: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html (default 'viridis')
# adding '_r' to the name uses the reversed colormap, e.g. 'Blues_r'

In [ ]:
vmin = merged_gdf['UC_date_number'].min()
vmax = merged_gdf['UC_date_number'].max()
cbar_ticks_locations = pd.np.linspace(vmin, vmax, 8)
cbar_ticks_labels = [pd.Timestamp(i).strftime('%b-%y') for i in cbar_ticks_locations]

In [ ]:
cbar = ax.figure.colorbar(
    plt.cm.ScalarMappable(norm=plt.Normalize(vmin=vmin, vmax=vmax), cmap=cmap), 
    cax=ax.figure.add_axes([0.76, 0.12, 0.03, 0.76]),  #[left, bottom, width, height] as fractions of figure width and height.
    ticks=cbar_ticks_locations
)
cbar.ax.set_yticklabels(cbar_ticks_labels)

ax.figure  # showing

### Save map as image

In [ ]:
# save as image
ax.figure.savefig('static_map.png', dpi=300, bbox_inches='tight')  # dpi is to increase the resolution of the image (default 200)

### Save merged data for later

In [ ]:
useful_cols = ['Province', 'LocalAuth', 'UC implem', 'UC_date_number', 'geometry']
merged_gdf['geometry'] = ensure_mutipoly(merged_gdf['geometry'])
merged_gdf[useful_cols].to_file("data/map_data.geojson", driver='GeoJSON')